In [1]:
# %load_ext autorelaod
# %autoreload 2

import sys
sys.path.append('./../RWKV-v4neo/src')
import torch 
from torch.utils import data
from enwik8_dataset import EnWik8
import json
import argparse
import os
from torch import nn
from lightning import seed_everything
seed_everything(42)

/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

In [2]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [3]:
data_path = '/system/user/beck/pwbeck/projects/rwkv/RWKV-LM/data/enwik8'
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

In [4]:
batch_size = 12

In [5]:
dataset = EnWik8(datafile=data_path)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=True)
args.vocab_size = dataset.vocab_size

Data has 99621832 tokens, 6064 vocab size.


In [6]:
batch = next(iter(dataloader))
x, y = batch
x.shape, y.shape

(torch.Size([12, 512]), torch.Size([12, 512]))

In [7]:
x

tensor([[16,  1,  1,  ...,  2, 84, 71],
        [ 2,  2,  2,  ..., 23, 25, 23],
        [20, 30, 17,  ..., 90, 67, 80],
        ...,
        [75, 80, 72,  ..., 67, 86,  2],
        [74, 86, 86,  ..., 53, 82, 67],
        [74, 67, 84,  ..., 86, 74, 71]])

### Reimplementation of RWKV-v4neo model

In [8]:
from rwkv_model import RWKV, RWKVConfig
from wkv_kernel import WKVConfig

In [9]:
config = RWKVConfig(embedding_dim=512,
                    attention_dim=512,
                    ffn_dim=2048,
                    num_blocks=6,
                    vocab_size=dataset.vocab_size, #6064
                    context_len=512, 
                    wkv_config=WKVConfig(float_mode='fp16'))


In [10]:
mymodel = RWKV(config).to(device='cuda')


Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv...


ninja: no work to do.


In [11]:
x = x.to(device='cuda')
out = mymodel(x)

In [12]:
out.shape

torch.Size([12, 512, 6064])

In [13]:
out

tensor([[[-3.3081e-01,  1.5812e-02,  4.8547e-01,  ...,  3.3276e-01,
           6.0093e-01,  6.1640e-01],
         [-3.6077e-02, -3.7648e-01, -1.8614e-01,  ...,  1.5427e-01,
          -1.2737e-02, -6.0445e-02],
         [ 4.3696e-02, -3.7088e-01, -2.7434e-01,  ..., -1.6589e-01,
          -2.8171e-02, -1.0600e-01],
         ...,
         [-5.0846e-01,  1.7093e-01, -4.8930e-01,  ...,  1.5211e-01,
           1.9901e-01, -6.4936e-01],
         [ 4.6809e-01,  1.6932e-01, -4.8769e-02,  ..., -6.1926e-01,
           5.0520e-01, -3.7347e-01],
         [-5.9225e-01, -2.5691e-02,  1.4321e-01,  ..., -3.3670e-01,
           6.9848e-01, -7.9805e-01]],

        [[-1.3204e-01,  3.9131e-01, -5.6200e-01,  ...,  2.1152e-01,
           3.2002e-01, -1.0070e+00],
         [-2.3457e-01,  4.8656e-01, -5.9861e-01,  ...,  3.2951e-01,
           4.6088e-01, -9.7160e-01],
         [-2.5510e-01,  3.9911e-01, -5.5693e-01,  ...,  2.7416e-01,
           4.8164e-01, -1.0298e+00],
         ...,
         [-2.5298e-01, -1

In [14]:
out[0,0,:]

tensor([-0.3308,  0.0158,  0.4855,  ...,  0.3328,  0.6009,  0.6164],
       device='cuda:0', grad_fn=<SliceBackward0>)